In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)

In [2]:
sample = ' if you want you'
# sample = ("if you want to build a ship, don't drum up people together to "
# "collect wood and don't assign them tasks and work, but rather "
# "teach them to long for the endless immensity of the sea.")

In [3]:
idx2char = list(set(sample))
char2idx = {c: i for i, c in enumerate(idx2char)}

# hyper parameter
dic_size = len(char2idx)
hidden_size = len(char2idx)
num_classes = len(char2idx)
batch_size = 1
sequence_length = len(sample) - 1
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]
x_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]

In [4]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])
    
# RNN
x_one_hot = tf.one_hot(X, num_classes)
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

In [5]:
# FC
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
# 원래는 outputs 를 logits 에 그대로 쓰는 거 아님
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X:x_data, Y:y_data})
        result = sess.run(prediction, feed_dict={X:x_data})
        
        result_str = [idx2char[c] for c in np.squeeze(result)]
        
        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 3.2174144 Prediction:                                                                                                                                                                                    
1 loss: 2.9646828 Prediction:                                                               o                             o                                                                                      
2 loss: 2.936613 Prediction: n ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
3 loss: 2.851164 Prediction:   t   tt t t  tt    t t e   t   t t    t  t      t     e  t  tt      tt   t t t   t t      t e        t t tt    t   t   e  t    et e     t    t   t e t       t         t  t e t   
4 loss: 2.748186 Prediction:                                                        e                                     e                                     e 

39 loss: 0.02977195 Prediction: f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
40 loss: 0.025292028 Prediction: f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
41 loss: 0.021643547 Prediction: f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
42 loss: 0.018302685 Prediction: f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
43 loss: 0.016038684 Prediction: f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but 